In [1]:
import pandas as pd
data = pd.read_csv('./Training_Set.csv')

In [15]:
from rdkit import Chem
from rdkit.Chem import rdmolops

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

from rdkit.Chem import AllChem
import numpy as np

In [16]:
def standardize_smiles_v2(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mol.UpdatePropertyCache(strict=False) # if strict is set to False, the method attempts to update the property cache for all atoms without raising an exception for unusual valences.
        # AssignStereochemistry: Assigns stereochemistry, ensuring that chiral centers are correctly identified and labeled.
        rdmolops.AssignStereochemistry(mol, cleanIt=True, force=True) # AssignStereochemistry: Assigns stereochemistry, ensuring that chiral centers are correctly identified and labeled.
        # Kekulization is the process of assigning single and double bond orders to the aromatic ring systems in a molecule, transforming the aromatic representation to a Kekule representation.
        Chem.SanitizeMol(mol, Chem.SanitizeFlags.SANITIZE_KEKULIZE, catchErrors=True) # catchErrors=True: This parameter means that if an error is encountered during the sanitization process, the function will return an error code rather than raising an exception. This can be useful if you want to handle errors more gracefully, for instance, by logging them and continuing with the next molecule.
        # converting a molecule back to a SMILES string to include information about isomerism
        # non-canonical SMILES string, which might not be unique for the given molecule.
        return Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    else:
        return None

In [98]:
data['Standard_SMILES'] = data['SMILES'].apply(standardize_smiles_v2)

In [18]:
# Function to generate ECFP4 1024-bit fingerprint
def generate_fp(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
        return list(fp)  # Convert to list to ensure proper format
    else:
        return [0] * nBits  # Return a vector of zeros if generation fails

In [19]:
data['ECFP4'] = data['Standard_SMILES'].apply(generate_fp)

In [104]:
data['ECFP4'] = data['ECFP4'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' ', dtype=int) if isinstance(x, str) else x)

In [ ]:
###############################
## AA Seq Encoding
###############################
from tensorflow.python.keras.preprocessing import sequence

seq_rdic = ['A','I','L','V','F','W','Y','N','C','Q','M','S','T','D','E','R','H','K','G','P','O','U','X','B','Z']
seq_dic = {w: i+1 for i,w in enumerate(seq_rdic)}

def encodeSeq(seq, seq_dic):
    if pd.isnull(seq):
        return [0]
    else:
        return [seq_dic[aa] for aa in seq]

In [ ]:
# Sequence encoding padding
df["Sequence_Encode"] = df.Sequence.map(lambda a: encodeSeq(a, seq_dic))
protein_feature = sequence.pad_sequences(df["Sequence_Encode"].values, maxlen=1200, padding='post', truncating='post', value=0)

df["Sequence_Encode"] = protein_feature.tolist()

In [106]:
# Prepare features and target. Convert to DataFrame
X_ecfp4 = np.array(data['ECFP4'].tolist())
X_seq = np.array(data['Sequence_Encode'].tolist())
y = data['Value'].values

#### Model using Keras

In [107]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import shap
from rdkit import Chem
from rdkit.Chem import Draw

In [108]:
# Set the visible GPU device to GPU 0
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

# Ensure GPU memory growth is set at the very beginning
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [109]:
# Split data into training and testing sets
X_ecfp4_train, X_ecfp4_test, X_seq_train, X_seq_test, y_train, y_test, train_index, test_index = train_test_split(
    X_ecfp4, X_seq, y, data.index, test_size=0.2, random_state=42)

In [110]:
# Define the model
input_ecfp4 = Input(shape=(X_ecfp4_train.shape[1],), name='ECFP4_Input')
input_seq = Input(shape=(X_seq_train.shape[1],), name='Seq_Input')

In [111]:
# First embedding layer for both inputs
x1 = Dense(128, activation='relu')(input_ecfp4)
x2 = Dense(128, activation='relu')(input_seq)

# Second embedding layer
x1 = Dense(64, activation='relu')(x1)
x2 = Dense(64, activation='relu')(x2)

# Concatenate both embeddings
x = Concatenate()([x1, x2])

# Third layer
x = Dense(32, activation='relu')(x)

# Output layer
output = Dense(1)(x)

# Create the model
model = Model(inputs=[input_ecfp4, input_seq], outputs=output)

In [112]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [113]:
# Train the model
model.fit([X_ecfp4_train, X_seq_train], y_train, epochs=500, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/500
4409/4409 [==============================] - 29s 7ms/step - loss: 0.9370 - val_loss: 0.6710
Epoch 2/500
4409/4409 [==============================] - 29s 7ms/step - loss: 0.5860 - val_loss: 0.5894
Epoch 3/500
4409/4409 [==============================] - 29s 7ms/step - loss: 0.4973 - val_loss: 0.5347
Epoch 4/500
4409/4409 [==============================] - 28s 6ms/step - loss: 0.4428 - val_loss: 0.4943
Epoch 5/500
4409/4409 [==============================] - 28s 6ms/step - loss: 0.4041 - val_loss: 0.4874
Epoch 6/500
4409/4409 [==============================] - 28s 6ms/step - loss: 0.3712 - val_loss: 0.4709
Epoch 7/500
4409/4409 [==============================] - 28s 6ms/step - loss: 0.3495 - val_loss: 0.4984
Epoch 8/500
4409/4409 [==============================] - 29s 7ms/step - loss: 0.3298 - val_loss: 0.5093
Epoch 9/500
4409/4409 [==============================] - 29s 7ms/step - loss: 0.3170 - val_loss: 0.4602
Epoch 10/500
4409/4409 [==============================] - 29s 6m

4409/4409 [==============================] - 32s 7ms/step - loss: 0.1557 - val_loss: 0.4470
Epoch 80/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1554 - val_loss: 0.4528
Epoch 81/500
4409/4409 [==============================] - 31s 7ms/step - loss: 0.1549 - val_loss: 0.4488
Epoch 82/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1546 - val_loss: 0.4430
Epoch 83/500
4409/4409 [==============================] - 31s 7ms/step - loss: 0.1538 - val_loss: 0.4434
Epoch 84/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1533 - val_loss: 0.4517
Epoch 85/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1528 - val_loss: 0.4441
Epoch 86/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1526 - val_loss: 0.4597
Epoch 87/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1521 - val_loss: 0.4520
Epoch 88/500
4409/4409 [==============================] - 32s 7ms/st

4409/4409 [==============================] - 32s 7ms/step - loss: 0.1296 - val_loss: 0.4742
Epoch 234/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1296 - val_loss: 0.4689
Epoch 235/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1300 - val_loss: 0.4681
Epoch 236/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1297 - val_loss: 0.4595
Epoch 237/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1293 - val_loss: 0.4663
Epoch 238/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1295 - val_loss: 0.4664
Epoch 239/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1304 - val_loss: 0.4802
Epoch 240/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1292 - val_loss: 0.4689
Epoch 241/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1293 - val_loss: 0.4761
Epoch 242/500
4409/4409 [==============================] - 3

4409/4409 [==============================] - 33s 7ms/step - loss: 0.1228 - val_loss: 0.4849
Epoch 388/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1232 - val_loss: 0.4846
Epoch 389/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1235 - val_loss: 0.4835
Epoch 390/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1232 - val_loss: 0.4924
Epoch 391/500
4409/4409 [==============================] - 33s 8ms/step - loss: 0.1231 - val_loss: 0.4766
Epoch 392/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1228 - val_loss: 0.4790
Epoch 393/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1230 - val_loss: 0.4816
Epoch 394/500
4409/4409 [==============================] - 33s 7ms/step - loss: 0.1225 - val_loss: 0.4802
Epoch 395/500
4409/4409 [==============================] - 32s 7ms/step - loss: 0.1233 - val_loss: 0.4768
Epoch 396/500
4409/4409 [==============================] - 3

In [114]:
# Predict on the test set
y_pred = model.predict([X_ecfp4_test, X_seq_test])

1378/1378 [==============================] - 4s 3ms/step


In [115]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
pcc, _ = pearsonr(y_test, y_pred.flatten())

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Pearson Correlation Coefficient (PCC):", pcc)

Mean Squared Error (MSE): 0.4778900664144314
Mean Absolute Error (MAE): 0.4423709812576498
Pearson Correlation Coefficient (PCC): 0.890083308979836


In [116]:
# Add prediction results to the test set dataframe
test_data = data.iloc[test_index].copy()
test_data['Prediction'] = y_pred

In [122]:
pearsonr(test_data['Value'], test_data['Prediction'])

(0.890083308979836, 0.0)

In [117]:
# Create the result dataframe
result_df = test_data[['Ikey', 'UNIPROT_AC', 'Value', 'Prediction']]